In [1]:
import xlrd
import os
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

Basic logic: 
1. Remove duplicates and cross-validate
2. Generate an output of all event extents (with 0/1 entries representing whether it belongs to NVivo categories)
3. Re-calculate events (#) and coverage (%)
4. Visualize patterns by collection
5. Visualize patterns by individual

# Process input

In [3]:
input_path = "/Users/yibingdu/VirtualBox Exchange/Interracial Event.xls"
input_path = "/Users/yibingdu/VirtualBox Exchange/Observations and Cultural References.xls"
input_path = "/Users/yibingdu/VirtualBox Exchange/Personal Experience.xls"
input_path = "/Users/yibingdu/VirtualBox Exchange/Policy; Politics; Legal.xls"
input_path = "/Users/yibingdu/VirtualBox Exchange/Second-hand Account.xls"
input_path = "/Users/yibingdu/VirtualBox Exchange/Sexual assault or rape.xls"
input_path = "/Users/yibingdu/VirtualBox Exchange/Sexual harassment.xls"
df = pd.read_excel(input_path)
df = df[["Name", "References", "Coverage"]]
df.drop_duplicates()
df.head()

,Name,References,Coverage
0,BWOH_Harrison_Pleasant,2,0.003135
1,BWOH_Hedgeman_Arnold,1,0.001507
2,BWSP_Black_Lois,1,0.005692
3,BWSP_Fairchild_Joyce,1,0.021519
4,BWSP_Gardner_Martha,3,0.074125


In [16]:
# interviewee name and collection
def extract_full_name(file_name):
    chunks = file_name.split("_")
    first_name = chunks[2:]
    for n in first_name:
        if len(n) == 1 and n.isalpha() != True:
            first_name.remove(n)
    return chunks[1] + ", " + " ".join(first_name)

def extract_collection(file_name):
    chunks = file_name.split("_")
    return chunks[0]

df["collection_id"] = df["Name"].apply(extract_collection)
df["name"] = df["Name"].apply(extract_full_name)
df = df[["collection_id", "name", "References", "Coverage"]]
df.head()

,collection_id,name,References,Coverage
0,BWOH,"Harrison, Pleasant",2,0.003135
1,BWOH,"Hedgeman, Arnold",1,0.001507
2,BWSP,"Black, Lois",1,0.005692
3,BWSP,"Fairchild, Joyce",1,0.021519
4,BWSP,"Gardner, Martha",3,0.074125


# Join with metadata

In [19]:
# process metadata
metadata = pd.read_csv("/Users/yibingdu/Downloads/Winter Quarter/OHTAP/metadata_cleaned.csv")
metadata = metadata[["collection_id", "name", "age", "birth_decade", "race", "education"]]
metadata.head()

,collection_id,name,age,birth_decade,race,education
0,BWOH,"Abbott, Jessie",80,1890,Black or African American,High school graduate
1,BWOH,"Adair, Christia",84,1890,Black or African American,Bachelor's degree
2,BWOH,"Adams, Frankie V.",75,1900,Black or African American,Graduate or professional degree
3,BWOH,"Adams, Kathleen Redding",86,1890,Black or African American,Associate's degree
4,BWOH,"Albrier, Frances Mary",81,1890,Black or African American,Bachelor's degree


In [25]:
# merge
merged = pd.merge(df, metadata, on = ["collection_id", "name"]) # see how many fail
merged.head()

,collection_id,name,References,Coverage,age,birth_decade,race,education
0,BWOH,"Harrison, Pleasant",2,0.003135,64,1910,Black or African American,9th to 12th grade but no diploma
1,BWOH,"Hedgeman, Arnold",1,0.001507,79,1890,Black or African American,Bachelor's degree
2,BWSP,"Black, Lois",1,0.005692,59,1930,White,Graduate or professional degree
3,BWSP,"Miller, Kristie",1,0.021092,61,1940,White,Graduate or professional degree
4,BWSP,"Palmer, Alison",4,0.012596,56,1930,White,Graduate or professional degree


In [26]:
# check if match
print(metadata.shape)
print(df.shape)
print(merged.shape)

(2734, 6)
(125, 4)
(100, 8)


# Visualization: by collection

# Visualization: by individual